# Phase transitions

## pyiron

In [7]:
import sys
sys.path.append("/cmmc/ptmp/smenon/FeC")

In [8]:
import os
from workflows.pyiron.build import bulk, repeat, change_composition
from workflows.pyiron.thermodynamics import calculate_free_energy
import matplotlib.pyplot as plt
from pyiron_workflow import Workflow

In [2]:
structure = bulk('Fe', crystalstructure='hcp', a=2.43).run()
structure = repeat(structure, (10,10,10)).run()

In [9]:
nstr = change_composition(structure, 'C', 0.2).run()

In [10]:
nstr.get_chemical_formula()

'C400Fe1600'

In [3]:
pair_style = "eam/alloy"
pair_coeff = "* * workflows/potentials/Fe_Ack.eam Fe"

In [4]:
wf = Workflow('fe1')
wf.bulk = bulk('Fe', cubic=True)
wf.structure = repeat(wf.bulk, (10,10,10))
wf.free_energy = calculate_free_energy(wf.structure, 
                                  pair_style, pair_coeff,
                                  reference_phase='solid',
                                  temperature=100,
                                  mode='pscale',
                                  n_switching_steps=50000,
                                  pressure = [120000, 150000],
                                  cores=1)

In [5]:
wf.run()

/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/ase/io/lammpsdata.py:51: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/executorlib/backend/interactive_serial.py", line 93, in <module>
    main(argument_lst=sys.argv)
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/executorlib/backend/interactive_serial.py", line 56, in main
    output = call_funct(input_dict=input_dict, funct=None, memory=memory)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/executorlib/standalone/interactive/backend.py", line 33, in call_funct
    return funct(input_dict["fn"], *input_dict["args"], **input_dict["kwargs"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/executorlib/standalone/i

## jobflow

In [5]:
from jobflow import job, run_locally, Flow
from workflows.jobflow.build import bulk, repeat
from workflows.jobflow.thermodynamics import calculate_free_energy

In [6]:
job_bulk = bulk('Fe', cubic=True)
job_structure = repeat(job_bulk.output, (5,5,5))
job_free_energy = calculate_free_energy(job_structure.output, 
                                  pair_style, pair_coeff,
                                  reference_phase='solid',
                                  temperature=600,
                                  cores=2)

In [7]:
flow = Flow([job_bulk, job_structure, job_free_energy])

In [8]:
run_locally(flow)

2025-09-26 13:55:20,176 INFO Started executing jobs locally
2025-09-26 13:55:20,327 INFO Starting job - bulk (ca726ab5-edf6-4a60-9cec-6a8c55bc4cbe)
2025-09-26 13:55:20,330 INFO Finished job - bulk (ca726ab5-edf6-4a60-9cec-6a8c55bc4cbe)
2025-09-26 13:55:20,330 INFO Starting job - repeat (49187356-c8fe-48b0-b1a0-d4f1a0616d72)
2025-09-26 13:55:20,355 INFO Finished job - repeat (49187356-c8fe-48b0-b1a0-d4f1a0616d72)
2025-09-26 13:55:20,355 INFO Starting job - calculate_free_energy (4b389fc1-596a-4093-bb52-20bfef32201d)


/home/menonsqr/miniforge3/envs/fec/lib/python3.12/site-packages/ase/io/lammpsdata.py:51: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(
--------------------------------------------------------------------------

  Local host:   login003
  Local device: hfi1_0
--------------------------------------------------------------------------
[login003:451103] 1 more process has sent help message help-mpi-btl-openib.txt / error in device init
[login003:451103] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
--------------------------------------------------------------------------

  Local host:   login003
  Local device: hfi1_0
--------------------------------------------------------------------------
[login003:451193] 1 more process has sent help message help-mpi-btl-openib.txt / error in device init
[login003:451193] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


2025-09-26 13:55:32,800 INFO Finished job - calculate_free_energy (4b389fc1-596a-4093-bb52-20bfef32201d)
2025-09-26 13:55:32,800 INFO Finished executing jobs locally


{'ca726ab5-edf6-4a60-9cec-6a8c55bc4cbe': {1: Response(output=Structure Summary
  Lattice
      abc : 2.87 2.87 2.87
   angles : 90.0 90.0 90.0
   volume : 23.639903
        A : 2.87 0.0 0.0
        B : 0.0 2.87 0.0
        C : 0.0 0.0 2.87
      pbc : True True True
  PeriodicSite: Fe (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
  PeriodicSite: Fe (1.435, 1.435, 1.435) [0.5, 0.5, 0.5], detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/menonsqr/repos/FeC'))},
 '49187356-c8fe-48b0-b1a0-d4f1a0616d72': {1: Response(output=Structure Summary
  Lattice
      abc : 14.350000000000001 14.350000000000001 14.350000000000001
   angles : 90.0 90.0 90.0
   volume : 2954.9878750000007
        A : 14.350000000000001 0.0 0.0
        B : 0.0 14.350000000000001 0.0
        C : 0.0 0.0 14.350000000000001
      pbc : True True True
  PeriodicSite: Fe (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
  PeriodicSite: Fe (1.435, 1.435, 1.435) [0.1, 0.1, 0.1]
  Peri